In [ ]:
# Bibliotecas necesarias para operaciones numéricas y análisis y manipulación de datos
import numpy as np
import pandas as pd

import openpyxl # Biblioteca para leer archivos de Excel

# Biblioteca para procesamiento de lenguaje natural y dividir texto en palabras.
import nltk
from nltk.tokenize import word_tokenize

# Biblioteca para crear y visualizar gráficos y datos estadísticos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Herramienta para convertir etiquetas en números, transformar y normalizar datos y
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential # Clase para construir modelos de redes neuronales
from tensorflow.keras import layers  # Módulo para definir capas en la red neuronal
from tensorflow.keras.optimizers import Adam  # Optimizador para ajustar la tasa de aprendizaje
from tensorflow.keras import metrics  # Módulo para métricas de evaluación en modelos de keras

In [ ]:
# Cambio el formato gráfico
pd.set_option("display.float_format", lambda x: "%.4f" % x)

# Lectura y primer acercamiento con las dimensiones del archivo
df = pd.read_csv("df_dates.csv")
df.shape

# Tipos de columnas numéricas y correlación entre ellas
numerics = ['int64', 'float64']
corr = df.select_dtypes(include=numerics).corr()

# Matriz sin repeticiones (triángulo inferior)
df_lt = corr.where(np.tril(np.ones(corr.shape)).astype(bool))

# Convertir la matriz a formato de texto, redondear y reemplazar ceros con cadenas vacías
df_lt_text = df_lt.applymap(lambda x: f'{x:.2f}' if pd.notna(x) and x != 0 else '')

<ipython-input-2-487ee36a7a65>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lt_text = df_lt.applymap(lambda x: f'{x:.2f}' if pd.notna(x) and x != 0 else '')


In [ ]:
# Crear la gráfica de calor
fig = px.imshow(df_lt, text_auto=False, aspect="auto", color_continuous_scale='haline_r', zmin=-1, zmax=1,
                template='plotly_dark', height=600, width=1200)

# Añadir un trazo alrededor de cada celda
fig.update_traces(hovertemplate=None, hoverinfo='skip', selector=dict(type='heatmap'), xgap=1, ygap=1, colorbar=dict(title='Value'))

# Añadir texto personalizado a las celdas
fig.update_traces(text=df_lt_text.values, texttemplate='%{text}', textfont=dict(size=12))

# Configurar el título y las etiquetas
fig.update_layout(title='<b>Matriz de Correlación de las Variables para la Regresión</b>', title_x=0.5, xaxis_title='Variable', yaxis_title='Variable')

# Mostrar la gráfica
fig.show()
df.isna().sum()
df.describe().T
df['alcaldia_hecho'].value_counts()

,count
alcaldia_hecho,
['cuauhtemoc'],280884
['iztapalapa'],280191
"['gustavo', 'a.', 'madero']",194092
"['benito', 'juarez']",153665
['coyoacan'],133161
"['alvaro', 'obregon']",130181
"['miguel', 'hidalgo']",122167
['tlalpan'],117095
"['venustiano', 'carranza']",110420


In [ ]:
# Pre-procesamiento
# Se importa el archivo con los folios (nums. tipo: 001, 002, ...) de las alcaldías de CDMX
boroughs_data_2020 = pd.read_excel("indices.xlsx")
boroughs_data_2020

# Información de la base y diccionario para mapear los valores del pre-procesamiento de la base de delitos a la base de folios
boroughs_data_2020.info()
borough_map = {
    "007": "['iztapalapa']",
    "005": "['gustavo', 'a.', 'madero']",
    "010": "['alvaro', 'obregon']",
    "012": "['tlalpan']",
    "003": "['coyoacan']",
    "015": "['cuauhtemoc']",
    "017": "['venustiano', 'carranza']",
    "013": "['xochimilco']",
    "014": "['benito', 'juarez']",
    "002": "['azcapotzalco']",
    "016": "['miguel', 'hidalgo']",
    "006": "['iztacalco']",
    "011": "['tlahuac']",
    "008": "['la', 'magdalena', 'contreras']",
    "004": "['cuajimalpa', 'de', 'morelos']",
    "009": "['milpa', 'alta']"
}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   folio_mun  16 non-null     int64  
 1   pob        16 non-null     int64  
 2   pob_nbi    16 non-null     int64  
 3   ids_ccevj  16 non-null     float64
 4   ids_csj    16 non-null     float64
 5   ids_caej   16 non-null     float64
 6   ids_ctelj  16 non-null     float64
 7   ids_cbdj   16 non-null     float64
 8   ids_rei    16 non-null     float64
 9   ids_cassi  16 non-null     float64
 10  ids_casi   16 non-null     float64
 11  idsm       16 non-null     float64
 12  e_idsm     16 non-null     object 
dtypes: float64(9), int64(3), object(1)
memory usage: 1.8+ KB


In [ ]:
# Se accede a la alcaldía con su folio
def get_borough(folio_mun):
    return borough_map.get(str(folio_mun)[-3:])

# Cambio de las alcaldías y nombres de columnas por unos más descriptivos
boroughs_data_2020["folio_mun"] = boroughs_data_2020["folio_mun"].astype(str).apply(lambda x: get_borough(x))
boroughs_data_2020.rename(columns={
    "pob": "Poblacion",
    "pob_nbi": "PoblacionPobreNBI",
    "ids_ccevj": "IDS_Vivienda",
    "ids_csj": "IDS_AdecSanitaria",
    "ids_caej": "IDS_AdecEnergetica",
    "ids_ctelj": "IDS_Telecomunicaciones",
    "ids_cbdj": "IDS_BienesDurables",
    "ids_rei": "IDS_Educacion",
    "ids_cassi": "IDS_SeguridadSocial",
    "ids_casi": "IDS_Salud",
    "idsm": "IDS",
    "e_idsm": "Estado_IDS"
    }, inplace=True)
boroughs_data_2020

# Conteo de las alcaldías para su validación
boroughs_data_2020['folio_mun'].value_counts()

,count
folio_mun,
['azcapotzalco'],1
['coyoacan'],1
"['cuajimalpa', 'de', 'morelos']",1
"['gustavo', 'a.', 'madero']",1
['iztacalco'],1
['iztapalapa'],1
"['la', 'magdalena', 'contreras']",1
"['milpa', 'alta']",1
"['alvaro', 'obregon']",1


In [ ]:
# Se agregan las columnas de indicadores sociales en función de las alcaldías registradas como ubicación de los delitos
df = pd.merge(df, boroughs_data_2020, how="left", left_on="alcaldia_hecho", right_on="folio_mun")

# Se elimina folio_mun, ya que sería repetir la columna de alcaldia_hecho
df = df.drop(columns=["folio_mun"])

# Visualización del nuevo df
df

# Organización y análisis de los delitos registrados por alcaldía y categoría del delito
crimes_borough = df.pivot_table(values = "categoria_delito", index = ["alcaldia_hecho"], columns = ["delito"], fill_value = 0,
                                margins = True, aggfunc = np.size)

# Lo separamos año por año, alcaldía por alcaldía
crimes_borough = crimes_borough.reset_index()

# Los ordenamos por alcaldía y año de incidencia
crimes_borough = crimes_borough.sort_values(by=['alcaldia_hecho']).reset_index(drop=True)
crimes_borough

# Tipos de datos en la base de delitos
df.dtypes

,0
fecha_hecho,object
hora_hecho,float64
delito,object
categoria_delito,object
colonia_hecho,object
alcaldia_hecho,object
latitud,float64
longitud,float64
tipo_delito,object
es_fin_de_semana,int64


In [ ]:
# Conversión de datos categóricos (alcaldías, colonias, categoría de delito)
numerics = ['int64', 'float64']
no_nums = []
L_E = LabelEncoder()

for dtype in df.dtypes:
    if dtype not in no_nums and dtype not in numerics:
        no_nums += [dtype]

# Se accede a las columnas no numéricas y, de encontrarse en aquellas variables de interés, se les convierte a numéricos
df_no_nums_cols = df.select_dtypes(include=no_nums).columns
for col in df_no_nums_cols:
    if col in ["delito", "categoria_delito", "colonia_hecho", "alcaldia_hecho", "tipo_delito", "Estado_IDS"]:
        df[col] = L_E.fit_transform(df[col])

# Correlación del dataset con variables numéricas
corr = df.select_dtypes(include=numerics).corr()

In [ ]:
# Matriz sin repeticiones (triángulo inferior)
df_lt = corr.where(np.tril(np.ones(corr.shape)).astype(bool))

# Convertir la matriz a formato de texto, redondear y reemplazar ceros con cadenas vacías
df_lt_text = df_lt.applymap(lambda x: f'{x:.2f}' if pd.notna(x) and x != 0 else '')

# Crear la gráfica de calor
fig = px.imshow(df_lt, text_auto=False, aspect="auto", color_continuous_scale='haline_r', zmin=-1, zmax=1,
                template='plotly_dark', height=900, width=1200)

# Añadir un trazo alrededor de cada celda
fig.update_traces(hovertemplate=None, hoverinfo='skip', selector=dict(type='heatmap'), xgap=1, ygap=1, colorbar=dict(title='Value'))

# Añadir texto personalizado a las celdas
fig.update_traces(text=df_lt_text.values, texttemplate='%{text}', textfont=dict(size=12))

# Configurar el título y las etiquetas
fig.update_layout(title='<b>Matriz de Correlación de las Variables para la Clasificación</b>', title_x=0.5, xaxis_title='Variable', yaxis_title='Variable')

# Mostrar la gráfica
fig.show()
#df

<ipython-input-8-1bd575a7042b>:5: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud,tipo_delito,es_fin_de_semana,...,IDS_Vivienda,IDS_AdecSanitaria,IDS_AdecEnergetica,IDS_Telecomunicaciones,IDS_BienesDurables,IDS_Educacion,IDS_SeguridadSocial,IDS_Salud,IDS,Estado_IDS
0,2016-01-01,0.0000,266,13,1238,8,19.3408,-99.1143,92,0,...,0.6080,0.8566,0.9995,0.8632,0.8461,0.8905,0.4998,0.6087,0.7462,1
1,2016-01-01,1.0000,131,0,326,8,19.3186,-99.0757,60,0,...,0.6080,0.8566,0.9995,0.8632,0.8461,0.8905,0.4998,0.6087,0.7462,1
2,2016-01-01,1.0000,38,0,1096,10,19.4572,-99.1725,22,0,...,0.7980,0.9716,0.9951,0.9338,0.9220,0.9459,0.6206,0.7625,0.8853,0
3,2016-01-01,1.0000,199,0,870,3,19.3127,-99.1118,92,0,...,0.7570,0.9504,0.9997,0.9230,0.9127,0.9386,0.6059,0.6905,0.8531,2
4,2016-01-01,2.0000,41,0,1408,1,19.4794,-99.1799,22,0,...,0.7522,0.9327,0.9996,0.9132,0.9077,0.9347,0.6734,0.7338,0.8575,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898445,2021-08-01,9.0000,2,0,1596,6,19.4828,-99.1151,2,1,...,0.6752,0.9145,0.9996,0.8793,0.8651,0.9130,0.5880,0.6728,0.8015,2
1898446,2024-05-27,10.0000,350,0,52,14,19.4184,-99.1162,109,0,...,0.7211,0.9520,0.9993,0.8950,0.8816,0.9249,0.5656,0.6531,0.8184,2
1898447,2024-03-08,0.0000,190,0,226,3,19.3442,-99.1547,86,0,...,0.7570,0.9504,0.9997,0.9230,0.9127,0.9386,0.6059,0.6905,0.8531,2
1898448,2024-03-07,0.0000,146,0,322,5,19.4344,-99.1430,66,0,...,0.7812,0.9769,0.9996,0.9097,0.8836,0.9347,0.5841,0.6727,0.8496,2


In [ ]:
df

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud,tipo_delito,es_fin_de_semana,...,IDS_Vivienda,IDS_AdecSanitaria,IDS_AdecEnergetica,IDS_Telecomunicaciones,IDS_BienesDurables,IDS_Educacion,IDS_SeguridadSocial,IDS_Salud,IDS,Estado_IDS
0,2016-01-01,0.0000,266,13,1238,8,19.3408,-99.1143,92,0,...,0.6080,0.8566,0.9995,0.8632,0.8461,0.8905,0.4998,0.6087,0.7462,1
1,2016-01-01,1.0000,131,0,326,8,19.3186,-99.0757,60,0,...,0.6080,0.8566,0.9995,0.8632,0.8461,0.8905,0.4998,0.6087,0.7462,1
2,2016-01-01,1.0000,38,0,1096,10,19.4572,-99.1725,22,0,...,0.7980,0.9716,0.9951,0.9338,0.9220,0.9459,0.6206,0.7625,0.8853,0
3,2016-01-01,1.0000,199,0,870,3,19.3127,-99.1118,92,0,...,0.7570,0.9504,0.9997,0.9230,0.9127,0.9386,0.6059,0.6905,0.8531,2
4,2016-01-01,2.0000,41,0,1408,1,19.4794,-99.1799,22,0,...,0.7522,0.9327,0.9996,0.9132,0.9077,0.9347,0.6734,0.7338,0.8575,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898445,2021-08-01,9.0000,2,0,1596,6,19.4828,-99.1151,2,1,...,0.6752,0.9145,0.9996,0.8793,0.8651,0.9130,0.5880,0.6728,0.8015,2
1898446,2024-05-27,10.0000,350,0,52,14,19.4184,-99.1162,109,0,...,0.7211,0.9520,0.9993,0.8950,0.8816,0.9249,0.5656,0.6531,0.8184,2
1898447,2024-03-08,0.0000,190,0,226,3,19.3442,-99.1547,86,0,...,0.7570,0.9504,0.9997,0.9230,0.9127,0.9386,0.6059,0.6905,0.8531,2
1898448,2024-03-07,0.0000,146,0,322,5,19.4344,-99.1430,66,0,...,0.7812,0.9769,0.9996,0.9097,0.8836,0.9347,0.5841,0.6727,0.8496,2


In [ ]:
# Exportación del nuevo archivo listo a un csv para los modelos
df.to_csv("df_completo.csv", index=False)